In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D,AveragePooling2D
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from tensorflow.keras.applications import EfficientNetB0,InceptionResNetV2,NASNetLarge
import os
import os.path
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop, SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import pandas as pd
import numpy as np
from tensorflow.keras import initializers
from tensorflow.keras.layers import BatchNormalization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip "/content/drive/MyDrive/Datasets/DatasetT2v2.zip"

Streaming output truncated to the last 5000 lines.
  inflating: dataset/club_sandwich858.jpg  
  inflating: dataset/churros5496.jpg  
  inflating: dataset/eggs_benedict15490.jpg  
  inflating: dataset/caesar_salad24537.jpg  
  inflating: dataset/club_sandwich680.jpg  
  inflating: dataset/chicken_wings17483.jpg  
  inflating: dataset/edamame9732.jpg  
  inflating: dataset/grilled_cheese_sandwich6456.jpg  
  inflating: dataset/clam_chowder10896.jpg  
  inflating: dataset/grilled_salmon17845.jpg  
  inflating: dataset/chicken_quesadilla14205.jpg  
  inflating: dataset/croque_madame19902.jpg  
  inflating: dataset/beef_carpaccio14550.jpg  
  inflating: dataset/breakfast_burrito5876.jpg  
  inflating: dataset/dumplings21206.jpg  
  inflating: dataset/bibimbap16724.jpg  
  inflating: dataset/french_fries20532.jpg  
  inflating: dataset/french_onion_soup22283.jpg  
  inflating: dataset/apple_pie12519.jpg  
  inflating: dataset/gnocchi2060.jpg  
  inflating: dataset/foie_gras248.jpg  
  infla

In [ ]:
df = pd.read_csv('/content/Food50T2.csv')
# df["Ingredients"]=df["Ingredients"].apply(lambda x:x.split(","))

In [ ]:
df.head()

FoodImage Ingredients
0           chicken_wings17261.jpg      baking
1            chicken_curry4204.jpg      baking
2              bruschetta11716.jpg    sauteing
3            caesar_salad24603.jpg      frying
4  grilled_cheese_sandwich6437.jpg      baking

In [ ]:
ingredients = []
all_ingredients =[]
classes = 0
with open("/content/CookingType.txt", 'r') as file:
    for line in file:
        line = line.rstrip('\n')
        ingredient_list = []
        for word in line.split(","):
            word = word.strip().lower()
            ingredient_list.append(word)
            all_ingredients.append(word)
        ingredients.append(ingredient_list)

all_ingredients = list(set(all_ingredients))
classes = len(all_ingredients)

In [ ]:
print(all_ingredients)
print(classes)

['baking', 'steaming', 'frozen', 'boiling', 'sauteing', 'frying', 'roasting']
7


In [ ]:
def append(filename):
    return "/content/dataset/"+filename
df["FoodImage"]=df["FoodImage"].apply(append)


In [ ]:
df.head()

FoodImage  sauteing  ...  frozen  roasting
0            /content/dataset/falafel6562.jpg         0  ...       0         0
1  /content/dataset/chocolate_mousse15920.jpg         0  ...       0         0
2       /content/dataset/french_toast3759.jpg         0  ...       0         0
3       /content/dataset/garlic_bread1599.jpg         0  ...       0         0
4     /content/dataset/croque_madame19631.jpg         0  ...       0         0

[5 rows x 8 columns]

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255)
validation_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen= ImageDataGenerator(rescale = 1./255)
cooking_type = ["sauteing", "frying", "steaming", "baking", "boiling", "frozen", "roasting"]

train_generator = train_datagen.flow_from_dataframe(dataframe=df[:20000],
                                                    directory=None,
                                                    x_col="FoodImage",
                                                    y_col=cooking_type,
                                                    batch_size=32,
                                                    color_mode = 'rgb',
                                                    shuffle=True,
                                                    validate_filenames=True,
                                                    class_mode="raw",
                                                    classes=all_ingredients,
                                                    target_size=(128,128))


validation_generator = validation_datagen.flow_from_dataframe(dataframe=df[20000:25000],
                                                            directory=None,
                                                            x_col="FoodImage",
                                                            y_col=cooking_type,
                                                            batch_size=32,
                                                            color_mode = 'rgb',
                                                            shuffle=True,    
                                                            validate_filenames=True,
                                                            class_mode="raw",
                                                            classes=all_ingredients,
                                                            target_size=(128,128))

Found 20000 validated image filenames.
Found 5000 validated image filenames.


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:220: UserWarning: `classes` will be ignored given the class_mode="raw"
  .format(self.class_mode))


In [ ]:
# ResNet transfer learning
base_model = ResNet152V2(
    include_top=False,
    weights="imagenet",
    input_shape=(128,128,3),
    pooling="Avg",)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

initializer = initializers.Zeros()

model = Sequential()
# model.add(base_model)

# model.add(Conv2D(2048, kernel_size=7, activation='relu', input_shape=(150,150,3)))
# model.add(MaxPooling2D(pool_size=(4,4), strides=(2,2)))
model.add(Conv2D(1024, kernel_size=7, activation='relu', input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(4,4), strides=(2,2)))
model.add(Conv2D(512, kernel_size=7, activation='relu', input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(4,4), strides=(2,2)))
model.add(Conv2D(256, kernel_size=7, activation='relu', input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(4,4), strides=(2,2)))
# model.add(Conv2D(128, kernel_size=7, activation='relu', input_shape=(150,150,3)))
# model.add(MaxPooling2D(pool_size=(4,4), strides=(2,2)))
# model.add(Conv2D(64, kernel_size=5, activation='relu'))
# model.add(MaxPooling2D(pool_size=(4,4), strides=(2,2)))

model.add(Flatten())
model.add(Dense(2048, kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(2048, kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(1024, kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(1024, kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(512, kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(512, kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(256, kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(256, kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(128, kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(128, kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))


model.add(Dense(classes, kernel_initializer=initializer))
model.add(Activation('softmax'))

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 144, 144, 2048)    303104    
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 71, 71, 2048)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 65, 65, 1024)      102761472 
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 31, 31, 1024)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 25, 25, 512)       25690624  
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 11, 11, 512)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 5, 5, 256)        

In [ ]:
checkpoint = ModelCheckpoint("./cooking_type_model.h5",
                             monitor="loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

# earlystop = EarlyStopping(monitor = 'accuracy', 
#                           min_delta = 0, 
#                           patience = 8,
#                           verbose = 1,
#                           restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'loss',
                              factor = 0.2,
                              patience = 1,
                              verbose = 1,
                              min_delta = 0.001)

In [ ]:
# we put our call backs into a callback list
# callbacks = [earlystop,checkpoint, reduce_lr]
callbacks = [checkpoint, reduce_lr]

# We use a very small learning rate Adam(learning_rate=0.001) SGD(learning_rate=0.1, momentum=0.9)
model.compile(loss = 'categorical_crossentropy',
              optimizer = SGD(learning_rate=0.001),
              metrics = ['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch = 625,
    epochs = 10,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = 156)

Epoch 1/10


In [ ]:
# multi-class classification with Keras
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
# load dataset
dataframe = pandas.read_csv("iris.data", header=None)
dataset = dataframe.values

FileNotFoundError: ignored

In [ ]:

X = dataset[:,0:4].astype(float)
Y = dataset[:,4]
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

# define baseline model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=4, activation='relu'))
	model.add(Dense(3, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))